### Step 1: Reading and preparing input file

In [ ]:
scores_file = 'Rescoring/Scores.csv'

df_vina = pd.read_csv('Sumario_VINA.csv',index_col=0)
df_vina.sort_values(['Ligand', 'Receptor'],inplace=True)
df_convex = pd.read_csv('Rescoring/Sumario_CONVEX.csv',index_col=0)
df_convex.sort_values(['Ligand', 'Receptor'],inplace=True)
df_convex[df_convex.columns[1:]] = df_convex[df_convex.columns[1:]].apply(pd.to_numeric)
df_ad4 = pd.read_csv('Rescoring/Sumario_ad4scoring.csv',index_col=1)
df_ad4 = df_ad4.drop(df_ad4.columns[0], axis=1)
df_ad4.sort_values(['Ligand', 'Receptor'],inplace=True)
df_ad4[df_ad4.columns[1:]] = df_ad4[df_ad4.columns[1:]].apply(pd.to_numeric)
df_vinardo = pd.read_csv('Rescoring/Sumario_vinardo.csv',index_col=1)
df_vinardo = df_vinardo.drop(df_vinardo.columns[0], axis=1)
df_vinardo.sort_values(['Ligand', 'Receptor'],inplace=True)
df_vinardo[df_vinardo.columns[1:]] = df_vinardo[df_vinardo.columns[1:]].apply(pd.to_numeric)
df_dkoes = pd.read_csv('Rescoring/Sumario_dkoes.csv',index_col=1)
df_dkoes = df_dkoes.drop(df_dkoes.columns[0], axis=1)
df_dkoes.sort_values(['Ligand', 'Receptor'],inplace=True)
df_dkoes[df_dkoes.columns[1:]] = df_dkoes[df_dkoes.columns[1:]].apply(pd.to_numeric)

vina_names = []
convex_names = []
ad4_names = []
vinardo_names = []
dkoes_names = []

vina_scores = []
convex_scores = []
ad4_scores = []
vinardo_scores = []
dkoes_scores = []

n = 3
for i in range(len(df_vina)):
    for j in range(n):
        vina_names.append(df_vina.index[i])
        convex_names.append(df_convex.index[i])
        ad4_names.append(df_ad4.index[i])
        vinardo_names.append(df_vinardo.index[i])
        dkoes_names.append(df_dkoes.index[i])
    vina_scores.extend(list(sorted(df_vina.iloc[i][1:]))[:3])
    convex_scores.extend(list(sorted(df_convex.iloc[i][1:]))[-3:])
    ad4_scores.extend(list(sorted(df_ad4.iloc[i][1:]))[:3])
    vinardo_scores.extend(list(sorted(df_vinardo.iloc[i][1:]))[:3])
    dkoes_scores.extend(list(sorted(df_dkoes.iloc[i][1:]))[:3])
     
Scores = pd.DataFrame({'Ligand':vina_names, 'Vina':vina_scores, 'Convex':convex_scores, 'Ad4':ad4_scores, 'Vinardo':vinardo_scores, 'Dkoes':dkoes_scores})
Scores = Scores.set_index('Ligand')
Scores.to_csv(scores_file)
Scores

### Step 2: Método Elbow Curve: Defeining number of Clusters

In [ ]:
pca = PCA()
pca = pca.fit(Scores[['Vina','Convex','Ad4', 'Dkoes', 'Vinardo']])
plt.plot(100-pca.explained_variance_)
plt.grid()
plt.show()

### Step 3: Using K-Means to select top Hits ligands

In [ ]:
scores_file = 'Rescoring/Scores.csv'
Scores = pd.read_csv(scores_file, index_col = 0)
km = KMeans(n_clusters = 3)
km_scores = km.fit_predict(Scores[['Vina','Convex','Ad4', 'Dkoes', 'Vinardo']])
Scores['Cluster'] = km_scores

#problematic = ['Decane','Sodium-dodecyl-sulfate', 'Berberine', '1-4-methylenetacrine', 'Velnacrine', 'Bis-tacrine', 'Bis-Acrilamida']
types = []

pos_dir = 'controles-positivos/'
positives = []
for l in os.listdir(pos_dir):
    posName = l[:-5]
    #if posName not in problematic:
        #positives.append(posName)

neg_dir = 'controles-negativos/'
negatives = []
for l in os.listdir(neg_dir):
    negName = l[:-5]
    #if negName not in problematic:
        #negatives.append(negName)

pp = Scores.loc[Scores.index.isin(positives)]
nn = Scores.loc[Scores.index.isin(negatives)]
#ss = Scores.loc[Scores.index.isin(problematic)]

for i in range(len(Scores)):
    if Scores.index[i] in positives:
        types.append(0)
    elif Scores.index[i] in negatives:
        types.append(1)
    else:
        types.append(2)
        
Scores['Type'] = types
#newScores = Scores.drop(problematic, axis=0)
#km_2 = KMeans(n_clusters = 2)
#km_scores2 = km_2.fit_predict(newScores[['Vina','Convex','Ad4', 'Dkoes', 'Vinardo']])
#newScores['newClusters'] = km_scores2

In [ ]:
### Step 4: Plotting 3-D image of Cl